In [1]:
#!/usr/bin/env python

import pandas as pd
import networkx as nx
from networkx.algorithms.bipartite.matrix import biadjacency_matrix
import numpy as np
from sklearn.metrics import precision_recall_curve, auc
from sklearn.preprocessing import normalize
import random
from sklearn import metrics
import time
from sklearn.decomposition import NMF
from scipy import sparse
# from libnmf.gnmf import GNMF

import numpy.linalg as LA

from scipy import stats

import matplotlib.pyplot as plt
# import math
# from scipy.linalg import logm

# from sklearn.metrics import confusion_matrix

from sklearn.linear_model import LinearRegression

# import seaborn as sns
from scipy import optimize
# from sklearn.metrics import r2_score
# from scipy.interpolate import make_interp_spline
# from sklearn.feature_selection import chi2
# import loess.loess_1d as l1d

# import tensorflow as tf

import multiprocessing
from joblib import Parallel, delayed
from math import sqrt
from sklearn.utils import parallel_backend

from itertools import product

from sklearn.metrics import roc_auc_score
from sklearn import svm

# from libsvm.svmutil import *  
# from libsvm.svm import *  
random.seed(1949) # for dataset split
np.random.seed(1949) # for matrix initialization

In [2]:
def option(str):
    global methodOption
    methodOption = str

In [3]:
def option2(tf):
    global normalizationOption
    normalizationOption = tf

In [4]:
def SVMloop(idx, X, X_new, y, y_new, Y_new, idx_test,c,g):
    # print(idx)
    # t = time.time()
    y_i = np.array(y[idx, :].copy()).tolist()
    if len(np.unique(y_i)) == 1:
        Y_new[idx, idx_test] = np.unique(y_i)
    else:

        if (methodOption == "SVMRBF1") | (methodOption == "SVMRBF2") |  (methodOption == "SVMRBF1&2"):
            svr = svm.SVR(kernel="rbf", C = c, gamma = g).fit(X, y_i)

        elif (methodOption == "SVML1") | (methodOption == "SVML2") |  (methodOption == "SVML1&2"):
            svr = svm.SVR(kernel="linear", C = c).fit(X, y_i)

        p_labels = svr.predict(X_new)
        Y_new[idx, idx_test] = p_labels

In [5]:
def SVMforAllSe(matrix, feature_matrix1, feature_matrix2, idx_train, idx_test, c, g):
    if (methodOption == "SVMRBF1") | (methodOption == "SVML1"):
        feature_matrix = feature_matrix1.copy()
    elif (methodOption == "SVMRBF2") | (methodOption == "SVML2"):
        feature_matrix = feature_matrix2.copy()
    elif (methodOption == "SVMRBF1&2") | (methodOption == "SVML1&2"):
        feature_matrix = np.array(pd.concat([pd.DataFrame(feature_matrix1), pd.DataFrame(feature_matrix2)], axis = 1))

    if normalizationOption == True:
        feature_matrix = (feature_matrix.copy() - feature_matrix.mean()) / feature_matrix.std()
    X = np.array(feature_matrix[idx_train, :].copy()).tolist()
    X_new = np.array(feature_matrix[idx_test, :].copy()).tolist()
    y = matrix[:, idx_train].copy()
    y_new = matrix[:, idx_test].copy()
    global Y_new
    Y_new = matrix.copy().astype(float)

    m,n = matrix.shape 

    
    # real_stdout = sys.stdout
    # sys.stdout = open(os.devnull, "w")

    
    with parallel_backend('threading'):
        Parallel(n_jobs=1)(delayed(SVMloop)(idx = i , X=X, X_new=X_new, y=y, y_new=y_new, Y_new=Y_new, \
            idx_test=idx_test,c=c,g=g) for i in range(m))
    

    # for i in range(m):
    #     y_i = np.array(y[i, :].copy()).tolist()
    #     y_new_i = np.array(y_new[i, :].copy()).tolist()

    #     model = svm_train(y_i, X, optionstr)
    #     p_labels, p_acc, p_vals = svm_predict(y_new_i, X_new, model)
        
    #     Y_new[i, idx_test] = p_labels

    # sys.stdout = real_stdout
    
    return Y_new

In [6]:
def FeaturePreprocess(df_all, drug_nodes):
    
    drug_nodes_df = np.intersect1d(df_all.index, drug_nodes)
    df = df_all.loc[drug_nodes_df]
    _, q = df.shape
    drug_nodes_diff = np.setdiff1d(drug_nodes, (df.index).tolist())
    n = len(drug_nodes_diff)
    df_diff = pd.DataFrame(np.zeros(n*q).reshape(n,q))
    df_diff.index = drug_nodes_diff
    df_diff.columns = df.columns
    df_all = pd.concat([df, df_diff], axis = 0)
    featureMat = df_all.loc[drug_nodes]
    return np.array(featureMat)

In [7]:
def network_preprocess(dir, drug_nodes_order):
    col_names = ["left_side","right_side","similairity"]
    df_drugs_sim = pd.read_csv(dir, sep =" ", names =col_names, header=None)

    source =df_drugs_sim["left_side"]
    destination = df_drugs_sim["right_side"]
    similarity = df_drugs_sim["similairity"]

    ###Drugs similarity Network###
    edge_list = zip(source,destination,similarity) # integrate 3 variables into tuple
    #print edge_list
    print("Side effect graph information loading...")
    G = nx.Graph() # creat a graph
    G.add_weighted_edges_from(edge_list) # add weighted edge from edge_list

    weight_matrix = nx.attr_matrix(G, edge_attr='weight', rc_order=drug_nodes_order)
    weight_matrix = np.array(weight_matrix)

    return weight_matrix

In [8]:
def pr_roc_curve(Ground_Truth, score):

    recall_fix = 0.2
    sort_idx = np.argsort(-score)
    Pre = score[sort_idx].copy()
    Gro = Ground_Truth[sort_idx].copy()
    ind = np.hstack(np.where(Gro > 0))
    thresholdList = np.unique(Pre[ind].copy())
    # print(thresholdList)
    if thresholdList.size == 0:
        Precision = np.array([0])
        Recall = np.array([0])
        TPR = np.array([0])
        FPR = np.array([0])
        F1 = np.array([0])
        TP = np.array([0])
        FP = np.array([sum(Pre != 0)])
        TN = np.array([sum(Pre == 0)])
        FN = np.array([0])
        return Precision, Recall, TPR, FPR, F1, TP, FP, TN, FN
    # thresholdList = np.hstack([min(Pre), thresholdList.copy(), max(Pre)])
    thresholdList = np.hstack(thresholdList.copy())
    thresholdList = np.unique(thresholdList)
    thresholdList = -np.sort(-thresholdList)
    AC_P = len(ind)
    AC_N = len(Gro) - AC_P
    N_thresholdList = len(thresholdList)

    
    TP = np.zeros(N_thresholdList)
    TN = np.zeros(N_thresholdList)
    FP = np.zeros(N_thresholdList)
    FN = np.zeros(N_thresholdList)


    N_pre = len(Pre)


    LOC = np.zeros(N_thresholdList)
    for i in range(len(thresholdList)):
        if np.isin(thresholdList[i], Pre):
            locs = np.hstack(np.where(thresholdList[i] == Pre))
            LOC[i] = locs[-1]
    LOC = LOC.astype(int)


    for i in range(len(LOC)):
        if i == 0:
            loc1 = 0
            loc2 = LOC[i]
            I = np.arange(loc1, loc2 + 1)
            J = np.arange(loc2 + 1, N_pre)
            TP[i] = len(np.hstack(np.where(Gro[I] == 1)))
            FP[i] = len(np.hstack(np.where(Gro[I] == 0)))

            TN[i] = len(np.hstack(np.where(Gro[J] == 0)))
            FN[i] = len(np.hstack(np.where(Gro[J] == 1)))
        else:
            loc1 = LOC[i - 1]
            loc2 = LOC[i]

            I = np.arange(loc1 + 1, loc2 + 1)

            TP[i] = TP[i - 1] + len(np.hstack(np.where(Gro[I] == 1)))
            FP[i] = FP[i - 1] + len(np.hstack(np.where(Gro[I] == 0)))

            TN[i] = TN[i - 1] - len(np.hstack(np.where(Gro[I] == 0)))
            FN[i] = FN[i - 1] - len(np.hstack(np.where(Gro[I] == 1)))

    TP = np.hstack([0, TP, AC_P])
    FP = np.hstack([0, FP, AC_N])
    TN = np.hstack([AC_N, TN, 0])
    FN = np.hstack([AC_P, FN, 0])

    # FPR = FP / AC_N
    # TPR = TP / AC_P
    FPR = np.zeros(N_thresholdList + 2)
    for i in range(N_thresholdList + 2):
        if TN[i] + FP[i] == 0 :
            FPR[i] = 0
        else:
            FPR[i] = FP[i] / (TN[i] + FP[i])

    # FPR = FP / (TN + FP)
    TPR = TP / (TP + FN)

    # Recall = TP / (TP + FN)
    Recall = TPR
    Precision = np.zeros(N_thresholdList + 2)
    # Precision = np.zeros(N_thresholdList)
    for i in range(N_thresholdList + 2):
        if TP[i] + FP[i] == 0 :
            Precision[i] = 0
        else:
            Precision[i] = TP[i] / (TP[i] + FP[i])
    # if (TP + FP== 0):
    #     Precision = np.zeros(N_thresholdList)
    # else:
    #     Precision = TP / (TP + FP)
    # jac=TP / (TP + FN + FP)
    # JAC = max(jac)
    # t = 2 / (1 / Recall + 1 / Precision)
    # score_F1 = max(t)

    # print('---TPR FPR')

    # print(TPR)
    # print(FPR)

    # print('---FP TN')

    # print(FP)
    # print(TN)

    # print('thlist')
    # print(thresholdList)
    # print(Pre)

    F1 = (2 * (Precision * Recall) / (Precision + Recall + 0.0000000000001)).max()



    return Precision, Recall, TPR, FPR, F1, TP, FP, TN, FN


In [9]:
def FmaxSmin(Ground_Truth_mat, score_mat, target_idx, TP, FP, TN, FN):
    m,n = Ground_Truth_mat.shape


    Ground_Truth = (Ground_Truth_mat[:, target_idx].copy()).ravel()
    score = (score_mat[:, target_idx].copy()).ravel()
    existing_drug_idx = np.setdiff1d(np.arange(n), target_idx)
    likelihood_obs = Ground_Truth_mat[:, existing_drug_idx].copy()
    ic = - np.log2((likelihood_obs.sum(axis=1) + 1)/(likelihood_obs.shape[1] + 2))

    sort_idx = np.argsort(-score)
    Pre = score[sort_idx].copy()
    Gro = Ground_Truth[sort_idx].copy()
    ind = np.hstack(np.where(Gro > 0))
    thresholdList = np.unique(Pre[ind].copy())
    # thresholdList = np.hstack([min(Pre), thresholdList.copy(), max(Pre)])
    thresholdList = np.hstack(thresholdList.copy())
    thresholdList = np.unique(thresholdList)
    thresholdList = -np.sort(-thresholdList)
    AC_P = len(ind)
    AC_N = len(Gro) - AC_P
    N_thresholdList = len(thresholdList)

    
    # TP = np.zeros(N_thresholdList)
    # TN = np.zeros(N_thresholdList)
    # FP = np.zeros(N_thresholdList)
    # FN = np.zeros(N_thresholdList)

    # m_t = np.zeros(N_thresholdList)
    n_e = np.zeros(N_thresholdList)


    icI_FN = np.zeros(N_thresholdList)
    icI_FP = np.zeros(N_thresholdList)

    icI_TP = np.zeros(N_thresholdList)

    Obs = Ground_Truth_mat[:, target_idx]
    for i in range(N_thresholdList):
        Pred = (score_mat[:, target_idx] > thresholdList[i])
        n_e[i] = sum((Pred.sum(axis = 1)) > 0)
        Result = Pred - Obs
        Result2 = Pred + Obs
        FP_col = (Result > 0)
        FN_col = (Result < 0)
        TP_col = (Result2 == 2)

        icI_FN[i] = sum(ic*(FN_col.sum(axis = 1)))
        icI_FP[i] = sum(ic*(FP_col.sum(axis = 1)))
        icI_TP[i] = sum(ic*(TP_col.sum(axis = 1)))

    n_e = np.hstack([0, n_e, m])

    AllN_icI_FN = sum(ic*(Obs.sum(axis = 1)))
    AllP_icI_FP = sum(ic*((1 - Obs).sum(axis = 1)))
    AllP_icI_TP = AllN_icI_FN


    icI_FN = np.hstack([AllN_icI_FN, icI_FN, 0])
    icI_FP = np.hstack([0, icI_FP, AllP_icI_FP])
    icI_TP = np.hstack([0, icI_TP, AllP_icI_TP])

    # FPR = FP / AC_N
    # TPR = TP / AC_P
    # FPR = FP / (TN + FP)
    # TPR = TP / (TP + FN)
    

    # Recall = TP / (TP + FN)
    # Recall = TPR
    Precision = np.zeros(N_thresholdList + 2)
    pr = np.zeros(N_thresholdList + 2)
    rc = np.zeros(N_thresholdList + 2)
    ru = np.zeros(N_thresholdList + 2)
    mi = np.zeros(N_thresholdList + 2)
    # Precision = np.zeros(N_thresholdList)
    for i in range(N_thresholdList + 2):
        if icI_TP[i] + icI_FP[i] == 0 :
            Precision[i] = 0
        else:
            Precision[i] = icI_TP[i] / (icI_TP[i] + icI_FP[i])

    pr = Precision
    rc = icI_TP / (icI_TP + icI_FN)

    for i in range(N_thresholdList + 2):
        if n_e[i] == 0 :
            # pr[i] = 0
            # rc[i] = 0
            ru[i] = 0
            mi[i] = 0
        else:
            # pr[i] = Precision[i] / n_e[i]
            # rc[i] = Recall[i] / n_e[i]

            ru[i] = icI_FN[i] / n_e[i]
            mi[i] = icI_FP[i] / n_e[i]
    

    # for i in range(N_thresholdList + 2):
    #     if n_e[i] == 0 :
    #         pr[i] = 0
    #         rc[i] = 0
    #         ru[i] = 0
    #         mi[i] = 0
    #     else:
    #         pr[i] = Precision[i]
    #         rc[i] = Recall[i]
    #         ru[i] = icI_FN[i]
    #         mi[i] = icI_FP[i]

    F = 2 * pr * rc / (pr + rc + 0.0000000000001)
    Fmax = max(F[n_e > 0])
    S = np.sqrt(ru**2 + mi**2)
    Smin = min(S[n_e > 0])

    return Fmax, Smin

In [10]:
def fold(IDX1,IDX2,feature_matrix1,feature_matrix2,matrix,c,g):
    # IDX1 target index, need to be evaluated
    # IDX2 test index, masked

    print('First few target index:', IDX1[0:10])
    print('First few mask index:', IDX2[0:10])

    target_idx = IDX1
    mask_idx = IDX2
    Ground_Truth = matrix.copy()
    side_effects_drug_relation_copy = matrix.copy()

    # target_idx = IDX2
    ### making all the links to predict as 0 ###############    
    for i in range(len(mask_idx)):
        side_effects_drug_relation_copy[:, mask_idx[i]] = 0
    
    m,n = side_effects_drug_relation_copy.shape

    drug_idx = list(range(n))
    existing_drug_idx = np.setdiff1d(drug_idx, mask_idx)
    
    # calculate the mean for each drug
    mean_side_effect_score = (Ground_Truth.copy()[:, existing_drug_idx]).mean(axis=1)
    score_mean = side_effects_drug_relation_copy.copy().astype(float)

    # Set the prediction into mean
    for i in range(m):
        score_mean[i, mask_idx] =  mean_side_effect_score[i]

    print(methodOption + ' starts:')
    # real_stdout = sys.stdout
    # sys.stdout = open(os.devnull, "w")
    side_effects_drug_relation_fact = SVMforAllSe(matrix=matrix, feature_matrix1=feature_matrix1, feature_matrix2=feature_matrix2, idx_train=existing_drug_idx, idx_test=target_idx, c=c, g=g)
    # sys.stdout = real_stdout
    print(methodOption + ' ends:')


    # Set the out put of GNMF as prediction score
    score = side_effects_drug_relation_fact.copy()
    # score = score_mean.copy()

    # Random score
    random_score = np.random.rand(m,n)
    
    pr_auc_all_mean = 0
    roc_auc_all_mean = 0
    F1_mean = 0
    pr_auc_per_drug_mean = 0
    roc_auc_per_drug_mean = 0
    fmax_mean = 0
    smin_mean = 0
    pr_auc_per_drug = 0
    roc_auc_per_drug = 0
    fmax = 0
    smin = 0
    F1 = 0
    pr_auc_all_rd = 0
    roc_auc_all_rd = 0
    pr_auc_per_drug_rd = 0
    roc_auc_per_drug_rd = 0
    fmax_rd = 0
    smin_rd = 0
    F1_rd = 0
    roc_auc_topn_rd = 0
    pr_auc_topn_rd = 0
    pr_auc_all = 0
    roc_auc_all = 0
    roc_auc_topn_mean = 0
    roc_auc_topn = 0
    pr_auc_topn_mean = 0
    pr_auc_topn = 0
    

    print("proportion of ground truth:", sum(Ground_Truth[:, target_idx].ravel())/(Ground_Truth[:, target_idx].shape[0]*Ground_Truth[:, target_idx].shape[1]))

    print('---evaluation---')
    prec, recall, threshold = precision_recall_curve(Ground_Truth[:, target_idx].ravel(), score[:, target_idx].ravel())
    pr_auc_all = auc(recall, prec)
    roc_auc_all = roc_auc_score(Ground_Truth[:, target_idx].ravel(), score[:, target_idx].ravel())



    print("-----")

    print("AUC-PR mean:", pr_auc_all_mean)
    # print("AUC-PR rd:", pr_auc_all_rd)
    print("AUC-PR all:", pr_auc_all)

    print("-----")

    print("AUC-ROC mean:", roc_auc_all_mean)
    # print("AUC-ROC rd:", roc_auc_all_rd)
    print("AUC-ROC all:", roc_auc_all)

    print("-----")

    print("AUC-PR per drug mean:", pr_auc_per_drug_mean)
    # print("AUC-PR per drug rd:", pr_auc_per_drug_rd)
    print("AUC-PR per drug:", pr_auc_per_drug)

    print("-----")

    print("AUC-ROC per drug mean:", roc_auc_per_drug_mean)
    # print("AUC-ROC per drug rd:", roc_auc_per_drug_rd)
    print("AUC-ROC per drug:", roc_auc_per_drug)

    print("-----")

    print("AUC-ROC top N mean:", roc_auc_topn_mean)
    # print("AUC-ROC top N rd:", roc_auc_topn_rd)
    print("AUC-ROC top N:", roc_auc_topn)

    print("-----")

    print("AUC-PR top N mean:", pr_auc_topn_mean)
    # print("AUC-PR top N rd:", pr_auc_topn_rd)
    print("AUC-PR top N:", pr_auc_topn)
    
    print("-----")

    print("F1 mean:", F1_mean)
    # print("F1 rd:", F1_rd)
    print("F1:", F1)

    print("-----")

    print("Fmax_mean", fmax_mean)
    # print("Fmax_rd", fmax_rd)
    print("Fmax", fmax)

    print("-----")

    print("Smin_mean", smin_mean)
    # print("Smin_rd", smin_rd)
    print("Smin", smin)



    return pr_auc_all_mean, roc_auc_all_mean, F1_mean, pr_auc_per_drug_mean, roc_auc_per_drug_mean, fmax_mean, smin_mean, \
        pr_auc_all, roc_auc_all, pr_auc_per_drug, roc_auc_per_drug, fmax, smin, F1, \
            pr_auc_all_rd, roc_auc_all_rd, pr_auc_per_drug_rd, roc_auc_per_drug_rd, fmax_rd, smin_rd, F1_rd, \
                roc_auc_topn_mean, roc_auc_topn_rd, roc_auc_topn, pr_auc_topn_mean, pr_auc_topn_rd, pr_auc_topn


In [11]:
def innerfold(IDX1,IDX2,feature_matrix1,feature_matrix2,matrix,c,g):
    # IDX1 target index, need to be evaluated
    # IDX2 test index, masked

    print('First few target index:', IDX1[0:10])
    print('First few mask index:', IDX2[0:10])

    target_idx = IDX1
    mask_idx = IDX2
    Ground_Truth = matrix.copy()
    side_effects_drug_relation_copy = matrix.copy()

    # target_idx = IDX2
    ### making all the links to predict as 0 ###############    
    for i in range(len(mask_idx)):
        side_effects_drug_relation_copy[:, mask_idx[i]] = 0
    
    m,n = side_effects_drug_relation_copy.shape

    drug_idx = list(range(n))
    existing_drug_idx = np.setdiff1d(drug_idx, mask_idx)
    
    # calculate the mean for each drug
    mean_side_effect_score = (Ground_Truth.copy()[:, existing_drug_idx]).mean(axis=1)
    score_mean = side_effects_drug_relation_copy.copy().astype(float)

    # Set the prediction into mean
    for i in range(m):
        score_mean[i, mask_idx] =  mean_side_effect_score[i]

    print(methodOption + ' starts:')
    # real_stdout = sys.stdout
    # sys.stdout = open(os.devnull, "w")
    side_effects_drug_relation_fact = SVMforAllSe(matrix=matrix, feature_matrix1=feature_matrix1, feature_matrix2=feature_matrix2, idx_train=existing_drug_idx, idx_test=target_idx, c=c, g=g)
    # sys.stdout = real_stdout
    print(methodOption + ' ends:')

    # Set the out put of GNMF as prediction score
    score = side_effects_drug_relation_fact.copy()

    # Random score
    random_score = np.random.rand(m,n)
    

    print("proportion of ground truth:", sum(Ground_Truth[:, target_idx].ravel())/(Ground_Truth[:, target_idx].shape[0]*Ground_Truth[:, target_idx].shape[1]))

    print('---evaluation---')



    pr_auc_all_mean = 0
    roc_auc_all_mean = 0
    F1_mean = 0
    pr_auc_per_drug_mean = 0
    roc_auc_per_drug_mean = 0
    fmax_mean = 0
    smin_mean = 0
    pr_auc_per_drug = 0
    roc_auc_per_drug = 0
    fmax = 0
    smin = 0
    F1 = 0
    pr_auc_all_rd = 0
    roc_auc_all_rd = 0
    pr_auc_per_drug_rd = 0
    roc_auc_per_drug_rd = 0
    fmax_rd = 0
    smin_rd = 0
    F1_rd = 0
    roc_auc_topn_rd = 0
    pr_auc_topn_rd = 0
    pr_auc_all = 0
    roc_auc_all = 0
    roc_auc_topn_mean = 0
    roc_auc_topn = 0
    pr_auc_topn_mean = 0
    pr_auc_topn = 0

    # time1 = time.time()
    

    # # ###### Evaluation all entries ######
    # ### GNMF
    # prec, recall, tpr, fpr, F1, TP, FP, TN, FN = pr_roc_curve(Ground_Truth[:, target_idx].ravel(), score[:, target_idx].ravel())
    # # Precision, Recall, TPR, FPR, F1, TP, FP, TN, FN
    # pr_auc_all = auc(recall, prec) 
    # roc_auc_all = auc(fpr, tpr)
    # # fmax, smin = FmaxSmin(Ground_Truth_mat=Ground_Truth, score_mat=score, target_idx=target_idx, TP = TP, FP = FP, TN = TN, FN = FN)

    # # ### Naive Model
    # prec, recall, tpr, fpr, F1_mean, TP, FP, TN, FN = pr_roc_curve(Ground_Truth[:, target_idx].ravel(), score_mean[:, target_idx].ravel())
    # pr_auc_all_mean = auc(recall, prec)
    # roc_auc_all_mean = auc(fpr, tpr)
    # # fmax_mean, smin_mean = FmaxSmin(Ground_Truth_mat=Ground_Truth, score_mat=score_mean, target_idx=target_idx, TP = TP, FP = FP, TN = TN, FN = FN)

    # print("our")
    # print("-----")

    # print("AUC-PR mean:", pr_auc_all_mean)
    # # print("AUC-PR rd:", pr_auc_all_rd)
    # print("AUC-PR all:", pr_auc_all)

    # print("-----")

    # print("AUC-ROC mean:", roc_auc_all_mean)
    # # print("AUC-ROC rd:", roc_auc_all_rd)
    # print("AUC-ROC all:", roc_auc_all)

    # print("-----")


    # time2 = time.time()
    # print("time for our AUC: ", time2 - time1)

    prec, recall, threshold = precision_recall_curve(Ground_Truth[:, target_idx].ravel(), score[:, target_idx].ravel())
    pr_auc_all = auc(recall, prec)
    roc_auc_all = roc_auc_score(Ground_Truth[:, target_idx].ravel(), score[:, target_idx].ravel())

    # prec, recall, threshold = precision_recall_curve(Ground_Truth[:, target_idx].ravel(), score_mean[:, target_idx].ravel())
    # pr_auc_all_mean = auc(recall, prec)
    # roc_auc_all_mean = roc_auc_score(Ground_Truth[:, target_idx].ravel(), score_mean[:, target_idx].ravel())

    # time3 = time.time()

    # print("sklearn")
    print("-----")

    # print("AUC-PR mean:", pr_auc_all_mean)
    # print("AUC-PR rd:", pr_auc_all_rd)
    print("AUC-PR all:", pr_auc_all)

    print("-----")

    # print("AUC-ROC mean:", roc_auc_all_mean)
    # # print("AUC-ROC rd:", roc_auc_all_rd)
    print("AUC-ROC all:", roc_auc_all)

    print("-----")
    # # print("time for sklearn AUC: ", time3 - time2)


    # ###### Evaluation per drug ######
    # ### GNMF
    # ALL_pr_auc = np.zeros(len(target_idx))
    # ALL_roc_auc = np.zeros(len(target_idx))
    # for i in range(len(target_idx)):
    #     prec, recall, tpr, fpr, _, _, _, _, _ = pr_roc_curve(Ground_Truth[:, target_idx[i]], score[:, target_idx[i]])
    #     ALL_pr_auc[i] = auc(recall, prec)
    #     ALL_roc_auc[i] = metrics.auc(fpr, tpr)  
    # pr_auc_per_drug = ALL_pr_auc.mean()
    # roc_auc_per_drug = ALL_roc_auc.mean()

    # ### Naive model
    # ALL_pr_auc = np.zeros(len(target_idx))
    # ALL_roc_auc = np.zeros(len(target_idx))
    # for i in range(len(target_idx)):
    #     prec, recall, tpr, fpr, _, _, _, _, _ = pr_roc_curve(Ground_Truth[:, target_idx[i]], score_mean[:, target_idx[i]])
    #     ALL_pr_auc[i] = auc(recall, prec)
    #     ALL_roc_auc[i] = metrics.auc(fpr, tpr)      
    # pr_auc_per_drug_mean = ALL_pr_auc.mean()
    # roc_auc_per_drug_mean = ALL_roc_auc.mean()

    

    # print("-----")

    # print("AUC-PR per drug mean:", pr_auc_per_drug_mean)
    # print("AUC-PR per drug:", pr_auc_per_drug)

    # print("-----")

    # print("AUC-ROC per drug mean:", roc_auc_per_drug_mean)
    # print("AUC-ROC per drug:", roc_auc_per_drug)

    # print("-----")





    return pr_auc_all_mean, roc_auc_all_mean, F1_mean, pr_auc_per_drug_mean, roc_auc_per_drug_mean, fmax_mean, smin_mean, \
        pr_auc_all, roc_auc_all, pr_auc_per_drug, roc_auc_per_drug, fmax, smin, F1, \
            pr_auc_all_rd, roc_auc_all_rd, pr_auc_per_drug_rd, roc_auc_per_drug_rd, fmax_rd, smin_rd, F1_rd, \
                roc_auc_topn_mean, roc_auc_topn_rd, roc_auc_topn, pr_auc_topn_mean, pr_auc_topn_rd, pr_auc_topn


In [12]:
def plotfold(IDX1,IDX2,feature_matrix1,feature_matrix2,matrix,c,g):
    # IDX1 target index, need to be evaluated
    # IDX2 test index, masked

    print('First few target index:', IDX1[0:10])
    print('First few mask index:', IDX2[0:10])

    target_idx = IDX1
    mask_idx = IDX2
    Ground_Truth = matrix.copy()
    side_effects_drug_relation_copy = matrix.copy()

    # target_idx = IDX2
    ### making all the links to predict as 0 ###############    
    for i in range(len(mask_idx)):
        side_effects_drug_relation_copy[:, mask_idx[i]] = 0
    
    m,n = side_effects_drug_relation_copy.shape

    drug_idx = list(range(n))
    existing_drug_idx = np.setdiff1d(drug_idx, mask_idx)
    
    # calculate the mean for each drug
    mean_side_effect_score = (Ground_Truth.copy()[:, existing_drug_idx]).mean(axis=1)
    score_mean = side_effects_drug_relation_copy.copy().astype(float)

    # Set the prediction into mean
    for i in range(m):
        score_mean[i, mask_idx] =  mean_side_effect_score[i]

    print(methodOption + ' starts:')
    # real_stdout = sys.stdout
    # sys.stdout = open(os.devnull, "w")
    side_effects_drug_relation_fact = SVMforAllSe(matrix=matrix, feature_matrix1=feature_matrix1, feature_matrix2=feature_matrix2, idx_train=existing_drug_idx, idx_test=target_idx, c=c, g=g)
    # sys.stdout = real_stdout
    print(methodOption + ' ends:')


    # Set the out put of GNMF as prediction score
    score = side_effects_drug_relation_fact.copy()
    # score = score_mean.copy()

    # Random score
    random_score = np.random.rand(m,n)
    
    pr_auc_all_mean = 0
    roc_auc_all_mean = 0
    F1_mean = 0
    pr_auc_per_drug_mean = 0
    roc_auc_per_drug_mean = 0
    fmax_mean = 0
    smin_mean = 0
    pr_auc_per_drug = 0
    roc_auc_per_drug = 0
    fmax = 0
    smin = 0
    F1 = 0
    pr_auc_all_rd = 0
    roc_auc_all_rd = 0
    pr_auc_per_drug_rd = 0
    roc_auc_per_drug_rd = 0
    fmax_rd = 0
    smin_rd = 0
    F1_rd = 0
    roc_auc_topn_rd = 0
    pr_auc_topn_rd = 0
    pr_auc_all = 0
    roc_auc_all = 0
    roc_auc_topn_mean = 0
    roc_auc_topn = 0
    pr_auc_topn_mean = 0
    pr_auc_topn = 0
    

    print("proportion of ground truth:", sum(Ground_Truth[:, target_idx].ravel())/(Ground_Truth[:, target_idx].shape[0]*Ground_Truth[:, target_idx].shape[1]))

    print('---evaluation---')

    prec, recall, prthreshold = precision_recall_curve(Ground_Truth[:, target_idx].ravel(), score[:, target_idx].ravel())
    pr_auc_all = auc(recall, prec)
    
    fpr, tpr, rocthreshold = metrics.roc_curve(Ground_Truth[:, target_idx].ravel(), score[:, target_idx].ravel())
    roc_auc_all = auc(fpr, tpr)


    print("-----")

    print("AUC-PR mean:", pr_auc_all_mean)
    # print("AUC-PR rd:", pr_auc_all_rd)
    print("AUC-PR all:", pr_auc_all)

    print("-----")

    print("AUC-ROC mean:", roc_auc_all_mean)
    # print("AUC-ROC rd:", roc_auc_all_rd)
    print("AUC-ROC all:", roc_auc_all)

    print("-----")

    print("AUC-PR per drug mean:", pr_auc_per_drug_mean)
    # print("AUC-PR per drug rd:", pr_auc_per_drug_rd)
    print("AUC-PR per drug:", pr_auc_per_drug)

    print("-----")

    print("AUC-ROC per drug mean:", roc_auc_per_drug_mean)
    # print("AUC-ROC per drug rd:", roc_auc_per_drug_rd)
    print("AUC-ROC per drug:", roc_auc_per_drug)

    print("-----")

    print("AUC-ROC top N mean:", roc_auc_topn_mean)
    # print("AUC-ROC top N rd:", roc_auc_topn_rd)
    print("AUC-ROC top N:", roc_auc_topn)

    print("-----")

    print("AUC-PR top N mean:", pr_auc_topn_mean)
    # print("AUC-PR top N rd:", pr_auc_topn_rd)
    print("AUC-PR top N:", pr_auc_topn)
    
    print("-----")

    print("F1 mean:", F1_mean)
    # print("F1 rd:", F1_rd)
    print("F1:", F1)

    print("-----")

    print("Fmax_mean", fmax_mean)
    # print("Fmax_rd", fmax_rd)
    print("Fmax", fmax)

    print("-----")

    print("Smin_mean", smin_mean)
    # print("Smin_rd", smin_rd)
    print("Smin", smin)


    Out1 = pd.DataFrame([prec, recall, prthreshold])
    Out2 = pd.DataFrame([fpr, tpr, rocthreshold])
    return Out1, Out2


In [13]:
def setvar_tune(size):
# set var for hyper pars tuning size is the hyper par size ALL_...
    global ALL_AUCPR_all_mean
    global ALL_AUROC_all_mean
    global ALL_AUCPR_per_drug_mean
    global ALL_AUROC_per_drug_mean
    global ALL_AUCPR_topn_mean
    global ALL_AUROC_topn_mean
    global ALL_F1_mean
    global ALL_Fmax_mean
    global ALL_Smin_mean

    global ALL_AUCPR_all_rd
    global ALL_AUROC_all_rd
    global ALL_AUCPR_per_drug_rd
    global ALL_AUROC_per_drug_rd
    global ALL_AUCPR_topn_rd
    global ALL_AUROC_topn_rd
    global ALL_F1_rd
    global ALL_Fmax_rd
    global ALL_Smin_rd

    global ALL_AUCPR_all
    global ALL_AUROC_all
    global ALL_AUCPR_per_drug
    global ALL_AUROC_per_drug
    global ALL_AUCPR_topn
    global ALL_AUROC_topn
    global ALL_F1
    global ALL_Fmax
    global ALL_Smin

    ALL_AUCPR_all_mean = np.zeros(size)
    ALL_AUROC_all_mean = np.zeros(size)
    ALL_AUCPR_per_drug_mean = np.zeros(size)
    ALL_AUROC_per_drug_mean = np.zeros(size)
    ALL_AUCPR_topn_mean = np.zeros(size)
    ALL_AUROC_topn_mean = np.zeros(size)
    ALL_F1_mean = np.zeros(size)
    ALL_Fmax_mean = np.zeros(size)
    ALL_Smin_mean = np.zeros(size)

    ALL_AUCPR_all_rd = np.zeros(size)
    ALL_AUROC_all_rd = np.zeros(size)
    ALL_AUCPR_per_drug_rd = np.zeros(size)
    ALL_AUROC_per_drug_rd = np.zeros(size)
    ALL_AUCPR_topn_rd = np.zeros(size)
    ALL_AUROC_topn_rd = np.zeros(size)
    ALL_F1_rd = np.zeros(size)
    ALL_Fmax_rd = np.zeros(size)
    ALL_Smin_rd = np.zeros(size)

    ALL_AUCPR_all = np.zeros(size)
    ALL_AUROC_all = np.zeros(size)
    ALL_AUCPR_per_drug = np.zeros(size)
    ALL_AUROC_per_drug = np.zeros(size)
    ALL_AUCPR_topn = np.zeros(size)
    ALL_AUROC_topn = np.zeros(size)
    ALL_F1 = np.zeros(size)
    ALL_Fmax = np.zeros(size)
    ALL_Smin = np.zeros(size)

In [14]:
def setvar_cv(FOLDS):
# set var for cv 
    global AUC_roc_all_mean
    global AUC_pr_all_mean
    global AUC_roc_per_drug_mean
    global AUC_pr_per_drug_mean
    global AUC_roc_topn_mean
    global AUC_pr_topn_mean
    global Fmax_mean
    global Smin_mean
    global F1_mean

    global AUC_roc_all
    global AUC_pr_all
    global AUC_roc_per_drug
    global AUC_pr_per_drug
    global AUC_roc_topn
    global AUC_pr_topn
    global Fmax
    global Smin
    global F1

    global AUC_roc_all_rd
    global AUC_pr_all_rd
    global AUC_roc_per_drug_rd
    global AUC_pr_per_drug_rd
    global AUC_roc_topn_rd
    global AUC_pr_topn_rd
    global Fmax_rd
    global Smin_rd
    global F1_rd
    
    AUC_roc_all_mean = np.zeros(FOLDS)
    AUC_pr_all_mean = np.zeros(FOLDS)
    AUC_roc_per_drug_mean = np.zeros(FOLDS)
    AUC_pr_per_drug_mean = np.zeros(FOLDS)
    AUC_roc_topn_mean = np.zeros(FOLDS)
    AUC_pr_topn_mean = np.zeros(FOLDS)
    Fmax_mean = np.zeros(FOLDS)
    Smin_mean = np.zeros(FOLDS)
    F1_mean = np.zeros(FOLDS)
    
    AUC_roc_all = np.zeros(FOLDS)
    AUC_pr_all = np.zeros(FOLDS)
    AUC_roc_per_drug = np.zeros(FOLDS)
    AUC_pr_per_drug = np.zeros(FOLDS)
    AUC_roc_topn = np.zeros(FOLDS)
    AUC_pr_topn = np.zeros(FOLDS)
    Fmax = np.zeros(FOLDS)
    Smin = np.zeros(FOLDS)
    F1 = np.zeros(FOLDS)

    AUC_roc_all_rd = np.zeros(FOLDS)
    AUC_pr_all_rd = np.zeros(FOLDS)
    AUC_roc_per_drug_rd = np.zeros(FOLDS)
    AUC_pr_per_drug_rd = np.zeros(FOLDS)
    AUC_roc_topn_rd = np.zeros(FOLDS)
    AUC_pr_topn_rd = np.zeros(FOLDS)
    Fmax_rd = np.zeros(FOLDS)
    Smin_rd = np.zeros(FOLDS)
    F1_rd = np.zeros(FOLDS)

In [15]:
def asgvar_tune(idx, results):
    # assign var for cv from results
    # f: size of hyper pars
    ALL_AUCPR_all_mean[idx] = results[0]
    ALL_AUROC_all_mean[idx] = results[1]
    ALL_F1_mean[idx] = results[2]
    ALL_AUCPR_per_drug_mean[idx] = results[3]
    ALL_AUROC_per_drug_mean[idx] = results[4]
    ALL_Fmax_mean[idx] = results[5]
    ALL_Smin_mean[idx] = results[6]
    ALL_AUCPR_all[idx] = results[7]
    ALL_AUROC_all[idx] = results[8]
    ALL_AUCPR_per_drug[idx] = results[9]
    ALL_AUROC_per_drug[idx] = results[10]
    ALL_Fmax[idx] = results[11]
    ALL_Smin[idx] = results[12]
    ALL_F1[idx] = results[13]
    ALL_AUCPR_all_rd[idx] = results[14]
    ALL_AUROC_all_rd[idx] = results[15]
    ALL_AUCPR_per_drug_rd[idx] = results[16]
    ALL_AUROC_per_drug_rd[idx] = results[17]
    ALL_Fmax_rd[idx] = results[18]
    ALL_Smin_rd[idx] = results[19]
    ALL_F1_rd[idx] = results[20]
    ALL_AUROC_topn_mean[idx] = results[21]
    ALL_AUROC_topn_rd[idx] = results[22]
    ALL_AUROC_topn[idx] = results[23]
    ALL_AUCPR_topn_mean[idx] = results[24]
    ALL_AUCPR_topn_rd[idx] = results[25]
    ALL_AUCPR_topn[idx] = results[26]

In [16]:
def asgvar_cv(f, results):
    # assign var for cv from results
    # f: size of hyper pars
    AUC_pr_all_mean[f] = results[0]
    AUC_roc_all_mean[f] = results[1]
    F1_mean[f] = results[2]
    AUC_pr_per_drug_mean[f] = results[3]
    AUC_roc_per_drug_mean[f] = results[4]
    Fmax_mean[f] = results[5]
    Smin_mean[f] = results[6]
    AUC_pr_all[f] = results[7]
    AUC_roc_all[f] = results[8]
    AUC_pr_per_drug[f] = results[9]
    AUC_roc_per_drug[f] = results[10]
    Fmax[f] = results[11]
    Smin[f] = results[12]
    F1[f] = results[13]
    AUC_pr_all_rd[f] = results[14]
    AUC_roc_all_rd[f] = results[15]
    AUC_pr_per_drug_rd[f] = results[16]
    AUC_roc_per_drug_rd[f] = results[17]
    Fmax_rd[f] = results[18]
    Smin_rd[f] = results[19]
    F1_rd[f] = results[20]
    AUC_roc_topn_mean[f] = results[21]
    AUC_roc_topn_rd[f] = results[22]
    AUC_roc_topn[f] = results[23]
    AUC_pr_topn_mean[f] = results[24]
    AUC_pr_topn_rd[f] = results[25]
    AUC_pr_topn[f] = results[26]

In [17]:
def tuning_plot(tuneVar, tune):
    if tune == None:
        return
    elif tune == "C":
        plt.figure()
        plt.plot(tuneVar, ALL_AUROC_all, color='navy', linewidth=0.5)
        plt.plot(tuneVar, ALL_AUROC_all_mean, color='red', linewidth=0.5)
        # plt.plot(tuneVar, ALL_AUROC_all_rd, color='black', linewidth=0.5)
        plt.xlabel("C")
        plt.ylabel("AUROC")
        plt.title('AUROC_all-C')
        plt.show()
        
        plt.figure()
        plt.plot(tuneVar, ALL_AUCPR_all, color='navy', linewidth=0.5)
        plt.plot(tuneVar, ALL_AUCPR_all_mean, color='red', linewidth=0.5)
        # plt.plot(tuneVar, ALL_AUCPR_all_rd, color='black', linewidth=0.5)
        plt.xlabel("C")
        plt.ylabel("AUPRC")
        plt.title('AUPRC_all-C')
        plt.show()     
        # per drug
        plt.figure()
        plt.plot(tuneVar, ALL_AUROC_per_drug, color='navy', linewidth=0.5)
        plt.plot(tuneVar, ALL_AUROC_per_drug_mean, color='red', linewidth=0.5)
        # plt.plot(tuneVar, ALL_AUROC_per_drug_rd, color='black', linewidth=0.5)
        plt.xlabel("C")
        plt.ylabel("AUROC")
        plt.title('AUROC_per_drug-C')
        plt.show()
        
        plt.figure()
        plt.plot(tuneVar, ALL_AUCPR_per_drug, color='navy', linewidth=0.5)
        plt.plot(tuneVar, ALL_AUCPR_per_drug_mean, color='red', linewidth=0.5)
        # plt.plot(tuneVar, ALL_AUCPR_per_drug_rd, color='black', linewidth=0.5)
        plt.xlabel("C")
        plt.ylabel("AUPRC")
        plt.title('AUPRC_per_drug-C')
        plt.show()
        # topn
        plt.figure()
        plt.plot(tuneVar, ALL_AUROC_topn, color='navy', linewidth=0.5)
        plt.plot(tuneVar, ALL_AUROC_topn_mean, color='red', linewidth=0.5)
        # plt.plot(tuneVar, ALL_AUROC_topn_rd, color='black', linewidth=0.5)
        plt.xlabel("C")
        plt.ylabel("AUROC")
        plt.title('AUROC_topn-C')
        plt.show()
        
        plt.figure()
        plt.plot(tuneVar, ALL_AUCPR_topn, color='navy', linewidth=0.5)
        plt.plot(tuneVar, ALL_AUCPR_topn_mean, color='red', linewidth=0.5)
        # plt.plot(tuneVar, ALL_AUCPR_topn_rd, color='black', linewidth=0.5)
        plt.xlabel("C")
        plt.ylabel("AUPRC")
        plt.title('AUPRC_topn-C')
        plt.show()
        # Fmax Smin
        plt.figure()
        plt.plot(tuneVar, ALL_Fmax, color='navy', linewidth=0.5)
        plt.plot(tuneVar, ALL_Fmax_mean, color='red', linewidth=0.5)
        # plt.plot(tuneVar, ALL_Fmax_rd, color='black', linewidth=0.5)
        plt.xlabel("C")
        plt.ylabel("Fmax")
        plt.title('Fmax-C')
        plt.show()
    
        plt.figure()
        plt.plot(tuneVar, ALL_Smin, color='navy', linewidth=0.5)
        plt.plot(tuneVar, ALL_Smin_mean, color='red', linewidth=0.5)
        # plt.plot(tuneVar, ALL_Smin_rd, color='black', linewidth=0.5)
        plt.xlabel("C")
        plt.ylabel("Smin")
        plt.title('Smin-C')
        plt.show()
    
        plt.figure()
        plt.plot(tuneVar, ALL_F1, color='navy', linewidth=0.5)
        plt.plot(tuneVar, ALL_F1_mean, color='red', linewidth=0.5)
        # plt.plot(tuneVar, ALL_F1_rd, color='black', linewidth=0.5)
        plt.xlabel("C")
        plt.ylabel("F1")
        plt.title('F1-C')
        plt.show()
    elif tune == "lmd":
        plt.figure()
        plt.plot(tuneVar, ALL_AUROC_all, color='navy', linewidth=0.5)
        plt.plot(tuneVar, ALL_AUROC_all_mean, color='red', linewidth=0.5)
        # plt.plot(tuneVar, ALL_AUROC_all_rd, color='black', linewidth=0.5)
        plt.xlabel("lambda")
        plt.ylabel("AUROC")
        plt.title('AUROC_all-lambda')
        plt.show()
        
        plt.figure()
        plt.plot(tuneVar, ALL_AUCPR_all, color='navy', linewidth=0.5)
        plt.plot(tuneVar, ALL_AUCPR_all_mean, color='red', linewidth=0.5)
        # plt.plot(tuneVar, ALL_AUCPR_all_rd, color='black', linewidth=0.5)
        plt.xlabel("lambda")
        plt.ylabel("AUPRC")
        plt.title('AUPRC_all-lambda')
        plt.show()     
        # per drug
        plt.figure()
        plt.plot(tuneVar, ALL_AUROC_per_drug, color='navy', linewidth=0.5)
        plt.plot(tuneVar, ALL_AUROC_per_drug_mean, color='red', linewidth=0.5)
        # plt.plot(tuneVar, ALL_AUROC_per_drug_rd, color='black', linewidth=0.5)
        plt.xlabel("lambda")
        plt.ylabel("AUROC")
        plt.title('AUROC_per_drug-lambda')
        plt.show()
        
        plt.figure()
        plt.plot(tuneVar, ALL_AUCPR_per_drug, color='navy', linewidth=0.5)
        plt.plot(tuneVar, ALL_AUCPR_per_drug_mean, color='red', linewidth=0.5)
        # plt.plot(tuneVar, ALL_AUCPR_per_drug_rd, color='black', linewidth=0.5)
        plt.xlabel("lambda")
        plt.ylabel("AUPRC")
        plt.title('AUPRC_per_drug-lambda')
        plt.show()  
        # per drug
        plt.figure()
        plt.plot(tuneVar, ALL_AUROC_topn, color='navy', linewidth=0.5)
        plt.plot(tuneVar, ALL_AUROC_topn_mean, color='red', linewidth=0.5)
        # plt.plot(tuneVar, ALL_AUROC_topn_rd, color='black', linewidth=0.5)
        plt.xlabel("lambda")
        plt.ylabel("AUROC")
        plt.title('AUROC_topn-lambda')
        plt.show()
        
        plt.figure()
        plt.plot(tuneVar, ALL_AUCPR_topn, color='navy', linewidth=0.5)
        plt.plot(tuneVar, ALL_AUCPR_topn_mean, color='red', linewidth=0.5)
        # plt.plot(tuneVar, ALL_AUCPR_topn_rd, color='black', linewidth=0.5)
        plt.xlabel("lambda")
        plt.ylabel("AUPRC")
        plt.title('AUPRC_topn-lambda')
        plt.show() 
        # Fmax Smin
        plt.figure()
        plt.plot(tuneVar, ALL_Fmax, color='navy', linewidth=0.5)
        plt.plot(tuneVar, ALL_Fmax_mean, color='red', linewidth=0.5)
        # plt.plot(tuneVar, ALL_Fmax_rd, color='black', linewidth=0.5)
        plt.xlabel("lambda")
        plt.ylabel("Fmax")
        plt.title('Fmax-lambda')
        plt.show()
    
        plt.figure()
        plt.plot(tuneVar, ALL_Smin, color='navy', linewidth=0.5)
        plt.plot(tuneVar, ALL_Smin_mean, color='red', linewidth=0.5)
        # plt.plot(tuneVar, ALL_Smin_rd, color='black', linewidth=0.5)
        plt.xlabel("lambda")
        plt.ylabel("Smin")
        plt.title('Smin-lambda')
        plt.show()
    
        plt.figure()
        plt.plot(tuneVar, ALL_F1, color='navy', linewidth=0.5)
        plt.plot(tuneVar, ALL_F1_mean, color='red', linewidth=0.5)
        # plt.plot(tuneVar, ALL_F1_rd, color='black', linewidth=0.5)
        plt.xlabel("lambda")
        plt.ylabel("F1")
        plt.title('F1-lambda')
        plt.show()
    elif tune == "component":
        plt.figure()
        plt.plot(tuneVar, ALL_AUROC_all, color='navy', linewidth=0.5)
        plt.plot(tuneVar, ALL_AUROC_all_mean, color='red', linewidth=0.5)
        # plt.plot(tuneVar, ALL_AUROC_all_rd, color='black', linewidth=0.5)
        plt.xlabel("k")
        plt.ylabel("AUROC")
        plt.title('AUROC_all-k')
        plt.show()
        
        plt.figure()
        plt.plot(tuneVar, ALL_AUCPR_all, color='navy', linewidth=0.5)
        plt.plot(tuneVar, ALL_AUCPR_all_mean, color='red', linewidth=0.5)
        # plt.plot(tuneVar, ALL_AUCPR_all_rd, color='black', linewidth=0.5)
        plt.xlabel("k")
        plt.ylabel("AUPRC")
        plt.title('AUPRC_all-k')
        plt.show()     
        # per drug
        plt.figure()
        plt.plot(tuneVar, ALL_AUROC_per_drug, color='navy', linewidth=0.5)
        plt.plot(tuneVar, ALL_AUROC_per_drug_mean, color='red', linewidth=0.5)
        # plt.plot(tuneVar, ALL_AUROC_per_drug_rd, color='black', linewidth=0.5)
        plt.xlabel("k")
        plt.ylabel("AUROC")
        plt.title('AUROC_per_drug-k')
        plt.show()
        
        plt.figure()
        plt.plot(tuneVar, ALL_AUCPR_per_drug, color='navy', linewidth=0.5)
        plt.plot(tuneVar, ALL_AUCPR_per_drug_mean, color='red', linewidth=0.5)
        # plt.plot(tuneVar, ALL_AUCPR_per_drug_rd, color='black', linewidth=0.5)
        plt.xlabel("k")
        plt.ylabel("AUPRC")
        plt.title('AUPRC_per_drug-k')
        plt.show()  
        # per drug
        plt.figure()
        plt.plot(tuneVar, ALL_AUROC_topn, color='navy', linewidth=0.5)
        plt.plot(tuneVar, ALL_AUROC_topn_mean, color='red', linewidth=0.5)
        # plt.plot(tuneVar, ALL_AUROC_topn_rd, color='black', linewidth=0.5)
        plt.xlabel("k")
        plt.ylabel("AUROC")
        plt.title('AUROC_topn-k')
        plt.show()
        
        plt.figure()
        plt.plot(tuneVar, ALL_AUCPR_topn, color='navy', linewidth=0.5)
        plt.plot(tuneVar, ALL_AUCPR_topn_mean, color='red', linewidth=0.5)
        # plt.plot(tuneVar, ALL_AUCPR_topn_rd, color='black', linewidth=0.5)
        plt.xlabel("k")
        plt.ylabel("AUPRC")
        plt.title('AUPRC_topn-k')
        plt.show() 
        # Fmax Smin
        plt.figure()
        plt.plot(tuneVar, ALL_Fmax, color='navy', linewidth=0.5)
        plt.plot(tuneVar, ALL_Fmax_mean, color='red', linewidth=0.5)
        # plt.plot(tuneVar, ALL_Fmax_rd, color='black', linewidth=0.5)
        plt.xlabel("k")
        plt.ylabel("Fmax")
        plt.title('Fmax-k')
        plt.show()
    
        plt.figure()
        plt.plot(tuneVar, ALL_Smin, color='navy', linewidth=0.5)
        plt.plot(tuneVar, ALL_Smin_mean, color='red', linewidth=0.5)
        # plt.plot(tuneVar, ALL_Smin_rd, color='black', linewidth=0.5)
        plt.xlabel("k")
        plt.ylabel("Smin")
        plt.title('Smin-k')
        plt.show()
    
        plt.figure()
        plt.plot(tuneVar, ALL_F1, color='navy', linewidth=0.5)
        plt.plot(tuneVar, ALL_F1_mean, color='red', linewidth=0.5)
        # plt.plot(tuneVar, ALL_F1_rd, color='black', linewidth=0.5)
        plt.xlabel("k")
        plt.ylabel("F1")
        plt.title('F1-k')
        plt.show()

In [18]:
def tuning_results(tuneVar):
    idx = np.argmax(ALL_AUCPR_all)
    Var = tuneVar[idx]
    Value = ALL_AUCPR_all[idx]

    # idx = np.argmax(ALL_AUCPR_per_drug)
    # Var = tuneVar[idx]
    # Value = ALL_AUCPR_per_drug[idx]
    print("best hyperpar: ", Var)
    print("AUPRC per drug: ", Value)

    
    # ALL_AUCPR_all[idx]
    # ALL_AUROC_all[idx]
    # ALL_AUCPR_per_drug[idx]
    # ALL_AUROC_per_drug[idx]
    # ALL_Fmax[idx]
    # ALL_Smin[idx]
    # ALL_F1[idx]
    return Var, Value

In [19]:
def setvar_besttune(innerfolds):
    global besttunevalue
    global besttunevar
    besttunevalue = np.zeros(innerfolds) # best metric value
    besttunevar = np.zeros(innerfolds) # the value of best var
    besttunevar = besttunevar.tolist()

In [20]:
def asg_besttune(f, value, var):
    besttunevalue[f] = value
    besttunevar[f] = var

In [21]:
def besttune():
    idx = np.argmax(besttunevalue)
    value = besttunevalue[idx]
    var = besttunevar[idx]
    return value, var

In [22]:
def cv_results():
    # print("Mean AUC-PR", AUC_test_pr.mean()," ", "Standard Deviation:", AUC_test_pr.std())
    # print("Mean AUC-ROC",AUC_test_roc.mean()," ", "Standard Deviation:", AUC_test_roc.std())
    # print("Mean NDCG:", ndcg_folds.mean(),"  " , " Standard Deviation:", ndcg_folds.std())
    
    print("Mean AUC_pr_all_mean:", AUC_pr_all_mean.mean()," ", "Standard Deviation:", AUC_pr_all_mean.std())
    print("Mean AUC_roc_all_mean", AUC_roc_all_mean.mean()," ", "Standard Deviation:", AUC_roc_all_mean.std())
    print("Mean AUC_pr_per_drug_mean", AUC_pr_per_drug_mean.mean()," ", "Standard Deviation:", AUC_pr_per_drug_mean.std())
    print("Mean AUC_roc_per_drug_mean", AUC_roc_per_drug_mean.mean()," ", "Standard Deviation:", AUC_roc_per_drug_mean.std())
    print("Mean AUC_pr_topn_mean", AUC_pr_topn_mean.mean()," ", "Standard Deviation:", AUC_pr_topn_mean.std())
    print("Mean AUC_roc_topn_mean", AUC_roc_topn_mean.mean()," ", "Standard Deviation:", AUC_roc_topn_mean.std())
    print("Mean Fmax_mean", Fmax_mean.mean()," ", "Standard Deviation:", Fmax_mean.std())
    print("Mean Smin_mean", Smin_mean.mean()," ", "Standard Deviation:", Smin_mean.std())
    print("Mean F1_mean", F1_mean.mean()," ", "Standard Deviation:", F1_mean.std())
    print("-----------")
    results_mean = np.array([AUC_pr_all_mean, AUC_roc_all_mean, AUC_pr_per_drug_mean, AUC_roc_per_drug_mean,\
        AUC_pr_topn_mean, AUC_roc_topn_mean, Fmax_mean, Smin_mean, F1_mean])

    print("Mean AUC_pr_all", AUC_pr_all.mean()," ", "Standard Deviation:", AUC_pr_all.std())
    print("Mean AUC_roc_all", AUC_roc_all.mean()," ", "Standard Deviation:", AUC_roc_all.std())
    print("Mean AUC_pr_per_drug", AUC_pr_per_drug.mean()," ", "Standard Deviation:", AUC_pr_per_drug.std())
    print("Mean AUC_roc_per_drug", AUC_roc_per_drug.mean()," ", "Standard Deviation:", AUC_roc_per_drug.std())
    print("Mean AUC_pr_topn", AUC_pr_topn.mean()," ", "Standard Deviation:", AUC_pr_topn.std())
    print("Mean AUC_roc_topn", AUC_roc_topn.mean()," ", "Standard Deviation:", AUC_roc_topn.std())
    print("Mean Fmax", Fmax.mean()," ", "Standard Deviation:", Fmax.std())
    print("Mean Smin", Smin.mean()," ", "Standard Deviation:", Smin.std())
    print("Mean F1", F1.mean()," ", "Standard Deviation:", F1.std())
    print("-----------")
    results = np.array([AUC_pr_all, AUC_roc_all, AUC_pr_per_drug, AUC_roc_per_drug,\
        AUC_pr_topn, AUC_roc_topn, Fmax, Smin, F1])

    print("Mean AUC_pr_all_rd", AUC_pr_all_rd.mean()," ", "Standard Deviation:", AUC_pr_all_rd.std())
    print("Mean AUC_roc_all_rd", AUC_roc_all_rd.mean()," ", "Standard Deviation:", AUC_roc_all_rd.std())
    print("Mean AUC_pr_per_drug_rd", AUC_pr_per_drug_rd.mean()," ", "Standard Deviation:", AUC_pr_per_drug_rd.std())
    print("Mean AUC_roc_per_drug_rd", AUC_roc_per_drug_rd.mean()," ", "Standard Deviation:", AUC_roc_per_drug_rd.std())
    print("Mean AUC_pr_topn_rd", AUC_pr_topn_rd.mean()," ", "Standard Deviation:", AUC_pr_topn_rd.std())
    print("Mean AUC_roc_topn_rd", AUC_roc_topn_rd.mean()," ", "Standard Deviation:", AUC_roc_topn_rd.std())
    print("Mean Fmax_rd", Fmax_rd.mean()," ", "Standard Deviation:", Fmax_rd.std())
    print("Mean Smin_rd", Smin_rd.mean()," ", "Standard Deviation:", Smin_rd.std())
    print("Mean F1_rd", F1_rd.mean()," ", "Standard Deviation:", F1_rd.std())
    return results_mean, results

In [23]:
def tuning_loop(innermatrix, idx_train_inner, idx_test_inner, feature_matrix_inner1, feature_matrix_inner2, hyperparList, i):
    
    c,g = hyperparList[i]
    idx_target_inner = idx_test_inner
    print('target size:', len(idx_target_inner))
    results = innerfold(idx_target_inner,idx_test_inner,feature_matrix1=feature_matrix_inner1,feature_matrix2=feature_matrix_inner2,matrix=innermatrix,c=c,g=g)
    asgvar_tune(i, results=results)
    print("------ C: ", c, "gamma: ", g, "------")

In [24]:
def main(method_option,normalization=True,Validation="nested_cv",sets="intersect", c=0, g=0):

    random.seed(1949) # for dataset split
    np.random.seed(1949) # for matrix initialization
    option(method_option)
    option2(normalization)

    random.seed(1949) # for dataset split
    np.random.seed(1949) # for matrix initialization
    df = pd.read_csv("data/side-effect-and-drug_name_upper.tsv",sep = "\t")
    drug_id = df["drugbank_id"] # put col of df in var
    drug_name = df["drugbank_name"]
    side_effect = df["side_effect_name"]
    
    
    edgelist1 = zip(side_effect, drug_name)
    ##making Biparite Graph##
    B = nx.DiGraph()
    B.add_nodes_from(side_effect,bipartite = 0)
    B.add_nodes_from(drug_name,bipartite = 1)
    B.add_edges_from(edgelist1)
    # B.add_weighted_edges_from(edgelist2)
    drug_nodes = {n for n, d in B.nodes(data=True) if d['bipartite']==1}
    side_effect_nodes = {n for n, d in B.nodes(data=True) if d['bipartite']==0}
    drug_nodes = list(drug_nodes)
    drug_nodes.sort()
    side_effect_nodes = list(side_effect_nodes)
    side_effect_nodes.sort()
    ###Getting the Bi-Adjacency matrix between side effects and drugs ###################
    matrix_all = biadjacency_matrix(B, row_order = side_effect_nodes, column_order = drug_nodes) # create biadjacency matrix for drug side effect graph
    matrix_all = matrix_all.A
    m_all,n_all = matrix_all.shape # number of side effect # number of drug
    
    
    ### Setting validation set / training set / testing set ###
    validate_sz = int(0.25 * n_all)
    IDX_all = list(range(n_all))
    random.shuffle(IDX_all)
    IDX_validate = sorted(IDX_all[0:validate_sz])
    print("first few validation set idx:")
    print(IDX_validate[0:10])
    IDX_validate_diff = np.setdiff1d(IDX_all, IDX_validate)
    matrix = matrix_all[:, IDX_validate_diff].copy()
    # featureMat1 = featureMat1_all[IDX_validate_diff, :][:, IDX_validate_diff].copy()
    # featureMat2 = featureMat2_all[IDX_validate_diff, :][:, IDX_validate_diff].copy()
    # print("WMK shape:")
    # print(featureMat1.shape)
    
    df1 = pd.read_csv("data/intersection_DGIdb_mat.tsv",sep = "\t")
    df2 = pd.read_csv("data/intersection_Fingerprint_mat.tsv",sep = "\t")
    featureMat1_all = FeaturePreprocess(df1, drug_nodes=drug_nodes)
    featureMat2_all = FeaturePreprocess(df2, drug_nodes=drug_nodes)
    # drug_nodes_feature1 = featureMat1_all.index
    # drug_nodes_feature2 = featureMat2_all.index
    featureMat1 = featureMat1_all[IDX_validate_diff, :].copy()
    featureMat2 = featureMat2_all[IDX_validate_diff, :].copy()
    
    
    non_zero_idx_union = np.hstack(np.where(~((featureMat1.sum(1) == 0) & (featureMat2.sum(1) == 0))))
    non_zero_idx_missing = np.hstack(np.where(~(~(featureMat1.sum(1) == 0) & ~(featureMat2.sum(1) == 0))))
    non_zero_idx_intersect = np.hstack(np.where(~(featureMat1.sum(1) == 0) & ~(featureMat2.sum(1) == 0)))
    if sets == "union":
        # union
        matrix = matrix[:, non_zero_idx_union].copy()
        featureMat1 = featureMat1[non_zero_idx_union, :].copy()
        featureMat2 = featureMat2[non_zero_idx_union, :].copy()
    elif sets == "intersect":
        # intersect
        non_zero_idx_intersect_all = np.hstack(np.where(~(featureMat1_all.sum(1) == 0) & ~(featureMat2_all.sum(1) == 0)))
    
        matrix_all = matrix_all[:, non_zero_idx_intersect_all].copy()
        featureMat1_all = featureMat1_all[non_zero_idx_intersect_all, :].copy()
        featureMat2_all = featureMat2_all[non_zero_idx_intersect_all, :].copy()
    
        matrix = matrix[:, non_zero_idx_intersect].copy()
        featureMat1 = featureMat1[non_zero_idx_intersect, :].copy()
        featureMat2 = featureMat2[non_zero_idx_intersect, :].copy()
    
        IDX_validate = np.setdiff1d(non_zero_idx_intersect_all, IDX_validate_diff)
        IDX_validate_diff = np.setdiff1d(non_zero_idx_intersect_all, IDX_validate)
    
        drug_nodes_intersect_all = np.array(drug_nodes)[non_zero_idx_intersect_all]
        drug_nodes_intersect_validate_diff = np.array(drug_nodes)[IDX_validate_diff]
        drug_nodes_intersect_validate = np.array(drug_nodes)[IDX_validate]
    
        IDX_validate = np.array([x for x in range(len(drug_nodes_intersect_all)) if drug_nodes_intersect_all[x] in drug_nodes_intersect_validate])
        IDX_validate_diff = np.array([x for x in range(len(drug_nodes_intersect_all)) if drug_nodes_intersect_all[x] in drug_nodes_intersect_validate_diff])
    

    m,n = matrix.shape # number of side effect # number of drug


    random.seed(1949) # for dataset split
    np.random.seed(1949) # for matrix initialization
    start_time = time.time()



    FOLDS = 5
    innerFOLDS = 4
    ####for test sets####
    setvar_cv(FOLDS)

    sz = n
    IDX = list(range(sz))
    fsz = int(sz/FOLDS)
    random.shuffle(IDX)
    IDX = np.array(IDX)
    offset = 0

    innersz = sz - fsz
    innerIDX = list(range(innersz))
    random.shuffle(innerIDX)
    innerIDX = np.array(innerIDX)
    innerfsz = int(innersz / innerFOLDS)
    inneroffset = 0
    # setvar_cv(FOLDS=FOLDS)

    if Validation == "nested_cv":
        for f in range(FOLDS):  # range(FOLDS):
            offset = 0 + fsz*f 
            idx_test = IDX[offset:offset + fsz]
    
            idx_train = IDX[np.setdiff1d(np.arange(len(IDX)), np.arange(offset,offset + fsz))]
            print("Fold:",f)
                
            innermatrix = matrix[:, idx_train]
    
            innerfeatureMat1 = featureMat1[idx_train, :].copy()
            innerfeatureMat2 = featureMat2[idx_train, :].copy()
            # if normalization == True:
            #     innerfeatureMat = (innerfeatureMat.copy() - innerfeatureMat.mean()) / innerfeatureMat.std()
            # print(type(weight_matrix1_inner))
    
            setvar_besttune(innerFOLDS)
    
            for innerf in range(innerFOLDS):
                inneroffset = 0 + innerf*innerfsz
                idx_test_inner = innerIDX[inneroffset:inneroffset + innerfsz]
                idx_train_inner = innerIDX[np.array(np.setdiff1d(np.arange(len(idx_train)), np.arange(inneroffset,inneroffset + innerfsz)))]
    
                print("Inner Fold:", innerf)


                if (methodOption == "SVMRBF1") |  (methodOption == "SVMRBF2") | (methodOption == "SVMRBF1&2"):
                    gamma = (10**np.arange(-3, -2, 1, dtype=float)).tolist()
                    C = (10**np.arange(-1, 1, 1, dtype=float)).tolist()
                elif (methodOption == "SVML1") | (methodOption == "SVML2") |  (methodOption == "SVML1&2"):
                    gamma = (10**np.arange(-3, -2, 1, dtype=float)).tolist()
                    C = (10**np.arange(-1, 0, 1, dtype=float)).tolist()
                hyperparList = list(product(C, gamma))
    
                setvar_tune(len(hyperparList))
    
                with parallel_backend('threading'):
                    Parallel(n_jobs=1)(delayed(tuning_loop)(innermatrix = innermatrix, idx_train_inner = idx_train_inner, \
                            idx_test_inner = idx_test_inner, feature_matrix_inner1 = innerfeatureMat1, feature_matrix_inner2 = innerfeatureMat2, \
                                hyperparList = hyperparList, i = i) \
                                    for i in range(len(hyperparList)))
    
                # tuning_plot(tuneVar=C, tune="C")
                hyperpars, evalValue = tuning_results(tuneVar=hyperparList)
    
    
    
                asg_besttune(innerf, value=evalValue, var=hyperpars)
                    
            _, bestHyperPars = besttune()
    
            print("--- tuning end ---")
            c, g = bestHyperPars
            idx_target = idx_test
            print('target size:', len(idx_target))
    
            print("------ C: ", c, "gamma: ", g, "------")
            # if normalization == True:
            #     featureMat = (featureMat.copy() - featureMat.mean()) / featureMat.std()
    
            results = fold(idx_target,idx_test,featureMat1,featureMat2,matrix,c=c,g=g)
            asgvar_cv(f=f, results=results)

        out_mean, out = cv_results()
        return out_mean, out

    elif Validation == "cv":
        # if normalization == True:
        #     featureMat = (featureMat.copy() - featureMat.mean()) / featureMat.std()
        
        setvar_besttune(FOLDS)

        for f in range(FOLDS):
            offset = 0 + fsz*f 
            idx_test = IDX[offset:offset + fsz]
            idx_train = IDX[np.setdiff1d(np.arange(len(IDX)), np.arange(offset,offset + fsz))]

            print("Fold:", f)

            if methodOption == "SVMRBF1" | methodOption == "SVMRBF2" |  methodOption == "SVMRBF1&2":
                gamma = (10**np.arange(-4, -1, 1, dtype=float)).tolist()
            elif methodOption == "SVML1" | methodOption == "SVML2" |  methodOption == "SVML1&2":
                gamma = (10**np.arange(1, 2, 1, dtype=float)).tolist()
            C = (10**np.arange(-1, -2, 1, dtype=float)).tolist()
            hyperparList = list(product(C, gamma))

            setvar_tune(len(hyperparList))
    
            with parallel_backend('threading'):
                Parallel(n_jobs=1)(delayed(tuning_loop)(innermatrix = matrix, idx_train_inner = idx_train, \
                        idx_test_inner = idx_test, feature_matrix_inner1 = featureMat1, feature_matrix_inner2 = featureMat2, \
                            hyperparList = hyperparList, i = i) \
                                    for i in range(len(hyperparList)))
    
            hyperpars, evalValue = tuning_results(tuneVar=hyperparList)
            asg_besttune(f, value=evalValue, var=hyperpars)

            # c, g = hyperpars
            # idx_target = idx_test
            # print('target size:', len(idx_target))

            # print("------ C: ", c, "gamma: ", g, "------")
            # # if normalization == True:
            # #     featureMat = (featureMat.copy() - featureMat.mean()) / featureMat.std()
    
            # results = fold(idx_target,idx_test,featureMat,matrix,c=c,g=g)
            # asgvar_cv(f=f, results=results)
    
        print("--- tuning end ---")
        # cv_results()
        _, bestHyperPars = besttune()
    elif Validation == "Validation":

        # validation
        idx_test = IDX_validate
        idx_train = IDX_validate_diff
        idx_target = idx_test
        print('target size:', len(idx_target))
        print("------ C: ", c, "gamma: ", g, "------")
        # if normalization == True:
        #     featureMat_all = (featureMat_all.copy() - featureMat_all.mean()) / featureMat_all.std()
        results = fold(idx_target,idx_test,featureMat1_all,featureMat2_all,matrix_all,c=c,g=g)
        return
    elif Validation == "plot":
        
        # validation
        idx_test = IDX_validate
        idx_train = IDX_validate_diff
        idx_target = idx_test
        print('target size:', len(idx_target))
        print("------ C: ", c, "gamma: ", g, "------")
        # if normalization == True:
        #     featureMat_all = (featureMat_all.copy() - featureMat_all.mean()) / featureMat_all.std()
        pr, roc = plotfold(idx_target,idx_test,featureMat1_all,featureMat2_all,matrix_all,c=c,g=g)
        return pr, roc


In [76]:
results_SVMRBF_mean, results_SVMRBF = main(method_option="SVMRBF1", Validation="nested_cv") # 0.1 0.001

first few validation set idx:
[2, 7, 12, 13, 17, 20, 26, 39, 45, 47]
Fold: 0
Inner Fold: 0
target size: 115
First few target index: [327  39 166 367  17 387 158  45 293  27]
First few mask index: [327  39 166 367  17 387 158  45 293  27]
SVMRBF1 starts:
SVMRBF1 ends:
proportion of ground truth: 0.02418288980175031
---evaluation---
-----
AUC-PR all: 0.3731193430394689
-----
AUC-ROC all: 0.8523367874645165
-----
------ C:  0.1 gamma:  0.0001 ------
best hyperpar:  (0.1, 0.0001)
AUPRC per drug:  0.3731193430394689
Inner Fold: 1
target size: 115
First few target index: [397 369  92 268 306 322 110 284 140 210]
First few mask index: [397 369  92 268 306 322 110 284 140 210]
SVMRBF1 starts:
SVMRBF1 ends:
proportion of ground truth: 0.024706275188892426
---evaluation---
-----
AUC-PR all: 0.3728342775339643
-----
AUC-ROC all: 0.8594936983887791
-----
------ C:  0.1 gamma:  0.0001 ------
best hyperpar:  (0.1, 0.0001)
AUPRC per drug:  0.3728342775339643
Inner Fold: 2
target size: 115
First few t

In [77]:
results_SVMRBF_mean, results_SVMRBF = main(method_option="SVMRBF2", Validation="nested_cv") # 0.1 0.001

first few validation set idx:
[2, 7, 12, 13, 17, 20, 26, 39, 45, 47]
Fold: 0
Inner Fold: 0
target size: 115
First few target index: [327  39 166 367  17 387 158  45 293  27]
First few mask index: [327  39 166 367  17 387 158  45 293  27]
SVMRBF2 starts:
SVMRBF2 ends:
proportion of ground truth: 0.02418288980175031
---evaluation---
-----
AUC-PR all: 0.33999345006946546
-----
AUC-ROC all: 0.8194723880962892
-----
------ C:  0.1 gamma:  0.0001 ------
target size: 115
First few target index: [327  39 166 367  17 387 158  45 293  27]
First few mask index: [327  39 166 367  17 387 158  45 293  27]
SVMRBF2 starts:
SVMRBF2 ends:
proportion of ground truth: 0.02418288980175031
---evaluation---
-----
AUC-PR all: 0.3719734720193777
-----
AUC-ROC all: 0.863806907814141
-----
------ C:  0.1 gamma:  0.001 ------
target size: 115
First few target index: [327  39 166 367  17 387 158  45 293  27]
First few mask index: [327  39 166 367  17 387 158  45 293  27]
SVMRBF2 starts:
SVMRBF2 ends:
proportion of

In [27]:
results_SVMRBF_mean, results_SVMRBF = main(method_option="SVMRBF1&2", Validation="nested_cv")

first few validation set idx:
[2, 7, 12, 13, 17, 20, 26, 39, 45, 47]
Fold: 0
Inner Fold: 0
target size: 115
First few target index: [327  39 166 367  17 387 158  45 293  27]
First few mask index: [327  39 166 367  17 387 158  45 293  27]
SVMRBF1&2 starts:
SVMRBF1&2 ends:
proportion of ground truth: 0.02418288980175031
---evaluation---
-----
AUC-PR all: 0.40649179515241585
-----
AUC-ROC all: 0.9065480764123905
-----
------ C:  0.1 gamma:  0.001 ------
target size: 115
First few target index: [327  39 166 367  17 387 158  45 293  27]
First few mask index: [327  39 166 367  17 387 158  45 293  27]
SVMRBF1&2 starts:
SVMRBF1&2 ends:
proportion of ground truth: 0.02418288980175031
---evaluation---
-----
AUC-PR all: 0.3879588971391534
-----
AUC-ROC all: 0.9081395536583743
-----
------ C:  1.0 gamma:  0.001 ------
best hyperpar:  (0.1, 0.001)
AUPRC per drug:  0.40649179515241585
Inner Fold: 1
target size: 115
First few target index: [397 369  92 268 306 322 110 284 140 210]
First few mask inde

In [ ]:
main(method_option="SVMRBF1",Validation="cv") # 0.1 0.001   

In [ ]:
main(method_option="SVMRBF2",Validation="cv") # 0.1 0.01

In [ ]:
main(method_option="SVMRBF1&2",Validation="cv") # 0.1 0.001

In [28]:
main(method_option="SVMRBF1", Validation="Validation", c=0.1, g=0.001) 

first few validation set idx:
[2, 7, 12, 13, 17, 20, 26, 39, 45, 47]
target size: 201
------ C:  0.1 gamma:  0.001 ------
First few target index: [ 4 11 14 27 32 34 35 37 41 47]
First few mask index: [ 4 11 14 27 32 34 35 37 41 47]
SVMRBF1 starts:
SVMRBF1 ends:
proportion of ground truth: 0.02256888445786783
---evaluation---
-----
AUC-PR mean: 0
AUC-PR all: 0.37778060148426956
-----
AUC-ROC mean: 0
AUC-ROC all: 0.8525988052321316
-----
AUC-PR per drug mean: 0
AUC-PR per drug: 0
-----
AUC-ROC per drug mean: 0
AUC-ROC per drug: 0
-----
AUC-ROC top N mean: 0
AUC-ROC top N: 0
-----
AUC-PR top N mean: 0
AUC-PR top N: 0
-----
F1 mean: 0
F1: 0
-----
Fmax_mean 0
Fmax 0
-----
Smin_mean 0
Smin 0


In [29]:
main(method_option="SVMRBF2", Validation="Validation", c=0.1, g=0.01) 

first few validation set idx:
[2, 7, 12, 13, 17, 20, 26, 39, 45, 47]
target size: 201
------ C:  0.1 gamma:  0.01 ------
First few target index: [ 4 11 14 27 32 34 35 37 41 47]
First few mask index: [ 4 11 14 27 32 34 35 37 41 47]
SVMRBF2 starts:
SVMRBF2 ends:
proportion of ground truth: 0.02256888445786783
---evaluation---
-----
AUC-PR mean: 0
AUC-PR all: 0.38073048188770975
-----
AUC-ROC mean: 0
AUC-ROC all: 0.8981263539903146
-----
AUC-PR per drug mean: 0
AUC-PR per drug: 0
-----
AUC-ROC per drug mean: 0
AUC-ROC per drug: 0
-----
AUC-ROC top N mean: 0
AUC-ROC top N: 0
-----
AUC-PR top N mean: 0
AUC-PR top N: 0
-----
F1 mean: 0
F1: 0
-----
Fmax_mean 0
Fmax 0
-----
Smin_mean 0
Smin 0


In [30]:
main(method_option="SVMRBF1&2", Validation="Validation", c=0.1, g=0.001) 

first few validation set idx:
[2, 7, 12, 13, 17, 20, 26, 39, 45, 47]
target size: 201
------ C:  0.1 gamma:  0.001 ------
First few target index: [ 4 11 14 27 32 34 35 37 41 47]
First few mask index: [ 4 11 14 27 32 34 35 37 41 47]
SVMRBF1&2 starts:
SVMRBF1&2 ends:
proportion of ground truth: 0.02256888445786783
---evaluation---
-----
AUC-PR mean: 0
AUC-PR all: 0.39633522520118214
-----
AUC-ROC mean: 0
AUC-ROC all: 0.8892180584060216
-----
AUC-PR per drug mean: 0
AUC-PR per drug: 0
-----
AUC-ROC per drug mean: 0
AUC-ROC per drug: 0
-----
AUC-ROC top N mean: 0
AUC-ROC top N: 0
-----
AUC-PR top N mean: 0
AUC-PR top N: 0
-----
F1 mean: 0
F1: 0
-----
Fmax_mean 0
Fmax 0
-----
Smin_mean 0
Smin 0


In [25]:
SVMRBF1_pr, SVMRBF1_roc = \
    main(method_option="SVMRBF1", Validation="plot", c=0.1, g=0.001)
SVMRBF1_pr.T.to_csv("Figs/SVMRBF1_pr.csv", index=False)
SVMRBF1_roc.T.to_csv("Figs/SVMRBF1_roc.csv", index=False)

first few validation set idx:
[2, 7, 12, 13, 17, 20, 26, 39, 45, 47]
target size: 201
------ C:  0.1 gamma:  0.001 ------
First few target index: [ 4 11 14 27 32 34 35 37 41 47]
First few mask index: [ 4 11 14 27 32 34 35 37 41 47]
SVMRBF1 starts:
SVMRBF1 ends:
proportion of ground truth: 0.02256888445786783
---evaluation---
-----
AUC-PR mean: 0
AUC-PR all: 0.37778060148426956
-----
AUC-ROC mean: 0
AUC-ROC all: 0.8525988052321316
-----
AUC-PR per drug mean: 0
AUC-PR per drug: 0
-----
AUC-ROC per drug mean: 0
AUC-ROC per drug: 0
-----
AUC-ROC top N mean: 0
AUC-ROC top N: 0
-----
AUC-PR top N mean: 0
AUC-PR top N: 0
-----
F1 mean: 0
F1: 0
-----
Fmax_mean 0
Fmax 0
-----
Smin_mean 0
Smin 0


In [26]:
SVMRBF2_pr, SVMRBF2_roc = \
    main(method_option="SVMRBF2", Validation="plot", c=0.1, g=0.01)
SVMRBF2_pr.T.to_csv("Figs/SVMRBF2_pr.csv", index=False)
SVMRBF2_roc.T.to_csv("Figs/SVMRBF2_roc.csv", index=False)

first few validation set idx:
[2, 7, 12, 13, 17, 20, 26, 39, 45, 47]
target size: 201
------ C:  0.1 gamma:  0.01 ------
First few target index: [ 4 11 14 27 32 34 35 37 41 47]
First few mask index: [ 4 11 14 27 32 34 35 37 41 47]
SVMRBF2 starts:
SVMRBF2 ends:
proportion of ground truth: 0.02256888445786783
---evaluation---
-----
AUC-PR mean: 0
AUC-PR all: 0.38073048188770975
-----
AUC-ROC mean: 0
AUC-ROC all: 0.8981263539903146
-----
AUC-PR per drug mean: 0
AUC-PR per drug: 0
-----
AUC-ROC per drug mean: 0
AUC-ROC per drug: 0
-----
AUC-ROC top N mean: 0
AUC-ROC top N: 0
-----
AUC-PR top N mean: 0
AUC-PR top N: 0
-----
F1 mean: 0
F1: 0
-----
Fmax_mean 0
Fmax 0
-----
Smin_mean 0
Smin 0


In [27]:
SVMRBF12_pr, SVMRBF12_roc = \
    main(method_option="SVMRBF1&2", Validation="plot", c=0.1, g=0.001)
SVMRBF12_pr.T.to_csv("Figs/SVMRBF12_pr.csv", index=False)
SVMRBF12_roc.T.to_csv("Figs/SVMRBF12_roc.csv", index=False)

first few validation set idx:
[2, 7, 12, 13, 17, 20, 26, 39, 45, 47]
target size: 201
------ C:  0.1 gamma:  0.001 ------
First few target index: [ 4 11 14 27 32 34 35 37 41 47]
First few mask index: [ 4 11 14 27 32 34 35 37 41 47]
SVMRBF1&2 starts:
SVMRBF1&2 ends:
proportion of ground truth: 0.02256888445786783
---evaluation---
-----
AUC-PR mean: 0
AUC-PR all: 0.39633522520118214
-----
AUC-ROC mean: 0
AUC-ROC all: 0.8892180584060216
-----
AUC-PR per drug mean: 0
AUC-PR per drug: 0
-----
AUC-ROC per drug mean: 0
AUC-ROC per drug: 0
-----
AUC-ROC top N mean: 0
AUC-ROC top N: 0
-----
AUC-PR top N mean: 0
AUC-PR top N: 0
-----
F1 mean: 0
F1: 0
-----
Fmax_mean 0
Fmax 0
-----
Smin_mean 0
Smin 0
